In [85]:
from tokenizers import Tokenizer

file_name = "sw_tokenizer.json"
tokenizer = Tokenizer.from_file(file_name)

In [86]:
text = "To be, or not to be:\n\nThat is \nthe question."

encoding = tokenizer.encode(text)
print(encoding.tokens)
decoding=tokenizer.decode(encoding.ids)
print(decoding)

['To', 'Ġbe', ',', 'Ġor', 'Ġnot', 'Ġto', 'Ġbe', ':', 'Ċ', 'Ċ', 'That', 'Ġis', 'Ġ', 'Ċ', 'the', 'Ġquestion', '.']
To be, or not to be:

That is 
the question.


In [87]:
import os
import torch
import torch.nn as nn
import torch.nn.functional as F

# Load a text file (any book / text)
with open("input.txt", "r", encoding="utf-8") as f:
    text = f.read()

encoding = tokenizer.encode(text)
print(len(encoding.ids))
data = torch.tensor(encoding.ids, dtype=torch.long)

317285


In [88]:
cpu_only=False
device = "cpu" if cpu_only or not torch.cuda.is_available() else "cuda"
print("device:",device)

device: cpu


In [89]:
n = int(0.9 * len(data))
train_data = data[:n]
val_data = data[n:]

In [90]:
block_size = 128   # context length
batch_size = 32

def get_batch(split):
    data_src = train_data if split == "train" else val_data
    ix = torch.randint(len(data_src) - block_size -1, (batch_size,))

    # Input tokens
    x = torch.stack([data_src[i:i+block_size] for i in ix])
    # Target = next character
    y = torch.stack([data_src[i+1:i+block_size+1] for i in ix])

    return x.to(device), y.to(device)

In [91]:
get_batch("train")

(tensor([[ 175, 1654,   11,  ...,    5,  314,  612],
         [2038, 2651,  327,  ...,  269, 5552,    7],
         [2904, 7291, 1083,  ...,   64, 6816,   98],
         ...,
         [  11,   64,  496,  ...,   64,  255,  117],
         [  64,  208,  515,  ...,   11,  341,  116],
         [ 445,  101,  328,  ...,   64,    4,  786]]),
 tensor([[1654,   11,   64,  ...,  314,  612,  487],
         [2651,  327, 1030,  ..., 5552,    7,   64],
         [7291, 1083,  178,  ..., 6816,   98,  165],
         ...,
         [  64,  496, 1205,  ...,  255,  117,   77],
         [ 208,  515,  148,  ...,  341,  116,  328],
         [ 101,  328,  170,  ...,    4,  786, 1099]]))

In [92]:
class CharEmbedding(nn.Module):
    def __init__(self, vocab_size, d_model, block_size):
        super().__init__()
        self.token_emb = nn.Embedding(vocab_size, d_model)
        self.pos_emb = nn.Embedding(block_size, d_model)

    def forward(self, x):
        B, T = x.shape
        tok = self.token_emb(x)              # (B, T, d_model)
        pos = self.pos_emb(torch.arange(T, device=device))  # (T, d_model)
        return tok + pos


In [93]:
class MultiHeadSelfAttention(nn.Module):
    def __init__(self, d_model, n_heads, block_size):
        super().__init__()
        assert d_model % n_heads == 0

        self.n_heads = n_heads
        self.d_head = d_model // n_heads

        self.key   = nn.Linear(d_model, d_model, bias=False)
        self.query = nn.Linear(d_model, d_model, bias=False)
        self.value = nn.Linear(d_model, d_model, bias=False)

        self.proj = nn.Linear(d_model, d_model)

        self.register_buffer(
            "mask",
            torch.tril(torch.ones(block_size, block_size)).bool()
        )

    def forward(self, x):
        B, T, C = x.shape

        # Project once
        K = self.key(x)    # (B, T, C)
        Q = self.query(x)
        V = self.value(x)

        # Split into heads
        K = K.view(B, T, self.n_heads, self.d_head).transpose(1, 2)
        Q = Q.view(B, T, self.n_heads, self.d_head).transpose(1, 2)
        V = V.view(B, T, self.n_heads, self.d_head).transpose(1, 2)
        # Shapes: (B, n_heads, T, d_head)

        # Attention scores
        att = (Q @ K.transpose(-2, -1)) / (self.d_head ** 0.5)
        # (B, n_heads, T, T)

        att = att.masked_fill(~self.mask[:T, :T], float('-inf'))
        att = F.softmax(att, dim=-1)

        # Weighted sum
        out = att @ V  # (B, n_heads, T, d_head)

        # Recombine heads
        out = out.transpose(1, 2).contiguous().view(B, T, C)

        return self.proj(out)

In [94]:
class TransformerBlock(nn.Module):
    def __init__(self, d_model, block_size, head_n):
        super().__init__()
        self.ln1 = nn.LayerNorm(d_model)
        self.attn = MultiHeadSelfAttention(d_model, head_n, block_size)
        self.ln2 = nn.LayerNorm(d_model)

        self.ff = nn.Sequential(
            nn.Linear(d_model, 4 * d_model),
            nn.ReLU(),
            nn.Linear(4 * d_model, d_model)
        )

    def forward(self, x):
        # Attention with residual
        x = x + self.attn(self.ln1(x))
        # Feed-forward with residual
        x = x + self.ff(self.ln2(x))
        return x

In [95]:
class SubwordLM(nn.Module):
    def __init__(self, vocab_size, d_model, block_size,head_n):
        super().__init__()
        self.embed = CharEmbedding(vocab_size, d_model, block_size)
        self.block = TransformerBlock(d_model,block_size, head_n)
        self.ln = nn.LayerNorm(d_model)
        self.head = nn.Linear(d_model, vocab_size)

    def forward(self, x, targets=None):
        x = self.embed(x)        # (B, T, d_model)
        x = self.block(x)
        x = self.ln(x)
        logits = self.head(x)   # (B, T, vocab_size)

        if targets is None:
            return logits

        B, T, V = logits.shape
        probs=logits.view(B*T, V) # probabilities: B * T, V
        ids = targets.view(B*T) # ids: B * T
        loss = F.cross_entropy(
            probs,
            ids
        )
        return logits, loss

In [96]:
model = SubwordLM(vocab_size, d_model=128, block_size=block_size, head_n=4).to(device)

def train():
    optimizer = torch.optim.AdamW(model.parameters(), lr=3e-4)
    for step in range(5000):
        xb, yb = get_batch("train")

        logits, loss = model(xb, yb)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if step % 500 == 0:
            print(f"step {step}, loss {loss.item():.4f}")

    torch.save(model.state_dict(), 'subword.pth')

model_path="subword.pth"
if os.path.exists(model_path):
    model.load_state_dict(torch.load(model_path, weights_only=True))
else:
    train()

step 0, loss 9.1311


KeyboardInterrupt: 

In [ ]:
@torch.no_grad()
def generate(model, start, max_new_tokens=200):
    model.eval()
    idx = torch.tensor([tokenizer.encode(start).ids], device=device, dtype=torch.long)

    for _ in range(max_new_tokens):
        idx_cond = idx[:, -block_size:]
        logits = model(idx_cond)
        logits = logits[:, -1, :]
        probs = F.softmax(logits, dim=-1)
        next_id = torch.multinomial(probs, num_samples=1)
        idx = torch.cat([idx, next_id], dim=1)

    return tokenizer.decode(idx[0].tolist())


In [ ]:
print(generate(model, "CORIOLANUS:"))